In [1]:
from quantileRegression import quantileRegression
import numpy as np
import runNumbers as rn
import Plotting as plot
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook
plt.style.use('ggplot')

Welcome to ROOTaaS 6.06/08


In [ ]:

#---------------------------------------------------------------------
#Prepare dataframes for the training.
#---------------------------------------------------------------------


In [ ]:
# Upload Dataframes:
qr_mc= quantileRegression("mc")
print "Load the mc dataframe"
startEvtmc = 0
stopEvtmc   = 2000000
print "Load the mc dataframe"
qr_mc.loadDF("/mnt/t3nfs01/data01/shome/mdonega/lxplus/work/CMSSW_8_0_26_patch1/src/diphotons/Analysis/macros/double_ele_v9_moriond17_mc/",
             "cicNoSigmaIetaIeta/trees/",
             ["DYJetsToLL_13TeV_EBHighR9", "DYJetsToLL_13TeV_EBLowR9", "DYJetsToLL_13TeV_EEHighR9", "DYJetsToLL_13TeV_EELowR9" ],
             startEvtmc, stopEvtmc, 12345)
#-------------------------------------------------------------
qr_data = quantileRegression("data")
startEvtdata = 0
stopEvtdata   = 2000000
print "Load the data dataframe"
qr_data.loadDF(" /mnt/t3nfs01/data01/shome/mdonega/lxplus/work/CMSSW_8_0_26_patch1/src/diphotons/Analysis/macros/double_ele_v9_moriond17_data/",
               "cicNoSigmaIetaIeta/trees/",
               ["Data_13TeV_EBHighR9", "Data_13TeV_EBLowR9", "Data_13TeV_EEHighR9", "Data_13TeV_EELowR9" ],
               startEvtdata, stopEvtdata, rn.run2016G[0],rn.run2016G[1])
dfdata = qr_data.getDF()
dfmc = qr_mc.getDF()

Load the mc dataframe
Load the mc dataframe
NT branches: 
Adding trees into a DataFrame
  adding  cicNoSigmaIetaIeta/trees/DYJetsToLL_13TeV_EBHighR9
  adding  cicNoSigmaIetaIeta/trees/DYJetsToLL_13TeV_EBLowR9
  adding  cicNoSigmaIetaIeta/trees/DYJetsToLL_13TeV_EEHighR9
  adding  cicNoSigmaIetaIeta/trees/DYJetsToLL_13TeV_EELowR9


In [ ]:
rv=np.arange(0,40,0.5)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Rho")
#plot histograms
plot.plotHistErr(dfdata.loc[:,"rho"], rv,Color="g")
plot.plotHistErr(dfmc.loc[:,"rho"], rv,Color="b",Weight=dfmc["weight"])
ax.set_title("Rho")

In [ ]:
#so the pileup is different for every run, let's do the ratio first:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Ratio_rho")

#Do the Ratio wrt the last 
plot.plotHistRatio(dfdata.loc[:,"rho"],dfmc.loc[:,"rho"],rv, Color="r",Weight2=dfmc["weight"])
ratiomc,edge,ratiomcErr= plot.getHistRatio(dfdata.loc[:,"rho"], dfmc.loc[:,"rho"],rv, Weight2=dfmc["weight"])

#ax.set_ylim([0,5])

ax.set_title('ratio')

In [ ]:
import pandas as pd
#find corrected value of correction:
dfmc['bin'] = np.digitize(dfmc['rho'],edge)
dfmc["corr"]= ratiomc[dfmc['bin'].values-1]
dfmc["weight"]=dfmc["weight"]*dfmc["corr"]

In [ ]:
#Check if now Rho Match
rv=np.arange(0,40,0.5)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Rho")
plot.plotHistErr(dfmc.loc[:,"rho"], rv,Color="b", Weight=dfmc["weight"])

plot.plotHistErr(dfdata.loc[:,"rho"], rv,Color="g",Weight=dfdata["weight"])
ax.set_title("Rho")

In [ ]:
#Check That R9 is still different
rv2 = np.arange(0.9,1.0,0.001)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("R9")
plot.plotHistErr(dfdata.loc[:,"R9"], rv2,Color="r",Weight=dfdata["weight"])
plot.plotHistErr(dfmc.loc[:,"R9"], rv2,Color="b",Weight=dfmc["weight"])
ax.set_title('R9')

In [ ]:
#set run_period on data given 
dfdata["runperiod"]=np.ones(len(dfdata.index))*4
dfmc["runperiod"]=np.ones(len(dfmc.index))*4

In [ ]:
import pandas as pd
#save the new montecarlo and the data in a h5 file:
dfname =  'df_mc_4.h5'
hdf = pd.HDFStore(dfname)
hdf.put('df', dfmc)
hdf.close()

dfname =  'df_data_4.h5'
hdf = pd.HDFStore(dfname)
hdf.put('df', dfdata)
hdf.close()
